# Math 243
**Harvard University**<br>

In [12]:
### Joseph Hostyk

### Math 243

### Setup of classes

import copy
import numpy as np
import random

class Graph(object):

    def __init__(self, numNodes):
        self.genotypes = []
        self.numNodes = numNodes
        # Weights is a matrix
        self.weights = None
        self.selectionMatrix = None
        self.genotypeCounts = {}

    def __str__(self):
        s = ""
        for i, geno in enumerate(self.genotypes):
            s += "{}. Neighbors: ".format(geno)
            for j in range(self.numNodes):
                if i != j and self.weights[i][j] != 0.0:
                    s += self.genotypes[j] + ", "
            s += "\n"
        return s


    # # Not sure if this is still necessary; maybe implement later.
    # def addNode(self, node, neighbors):
    #     return

    # Helpful to just have the frequencies in a dict. Call this after
    # the genotypes array is initialized.
    def calculateGenotypeFrequencies(self):
        for g in self.genotypes:
            if g not in self.genotypeCounts:
                self.genotypeCounts[g] = 0
            self.genotypeCounts[g] += 1

    def replaceNode(self, oldIndex, newGenotype):
        self.genotypeCounts[self.genotypes[oldIndex]] -= 1
        self.genotypes[oldIndex] = newGenotype
        if newGenotype not in self.genotypeCounts:
            self.genotypeCounts[newGenotype] = 0
        self.genotypeCounts[newGenotype] += 1        


    # def calculateTotalFitness(self):
    #     totalFitness = 0.0
    #     for node in self.graph:
    #         totalFitness += node.fitness
    #     self.totalFitness = totalFitness
    #     return totalFitness

    # def updateTotalFitness(self, difference):
    #     self.totalFitness += difference

class Lattice(Graph):

    def __init__(self, rows, cols):
        Graph.__init__(self, rows*cols)
        self.numRows = rows
        self.numCols = cols
        self.weights = [[0]*self.numNodes for i in range(self.numNodes)]

        self.genotypes = ["AA"]*(rows*cols)
        self.calculateGenotypeFrequencies()

        for r in range(rows):
            for c in range(cols):
                neighbors = []
                # Being careful of the edges:
                if r != 0:
                    self.weights[r*rows+c][(r-1)*rows+c] = 1
                if r != rows -1 :
                    self.weights[r*rows+c][(r+1)*rows+c] = 1
                if c != 0:
                    self.weights[r*rows+c][r*rows+c -1] = 1
                if c != cols - 1:
                    self.weights[r*rows+c][r*rows+c + 1] = 1

    def __str__(self):
        s = ""
        for r in range(self.numRows):
            for c in range(self.numCols):
                s += self.genotypes[r*c-1] + " "
            s += "\n"
        return s

class FullyConnected(Graph):
    def __init__(self, numNodes):
        Graph.__init__(self, numNodes)
        self.weights = [[1]*self.numNodes for i in range(self.numNodes)]
        self.genotypes = ["AA"]*(numNodes)        
        self.calculateGenotypeFrequencies()


class Bipartite(Graph):
    def __init__(self, numNodes, fitness, deathrate, genotype):
        Graph.__init__(self)

In [13]:
### driveOnGraphs: runs the simulations

import copy
import sys
import numpy as np

from graphs import *

# from plotlySignIn import *

# The probability that a Driven gene produces a Driven offspring
P = 1.0

fitness = {"AA": 1.0, "AD": 1.0, "DD": 1.0}
deathRates = {"AA": 1.0, "AD": 1.0, "DD": 1.0}



def findTotalFitness(genotypes):
    totalFitness = 0.0
    for g in genotypes:
        totalFitness += fitness[g]
    return totalFitness

# What offspring will be produced by A and B?
def matingOutcome(A, B):

    possibleOffspring = ["DD", "AD", "AA"]

    # We use Chuck's mating-tables to find the probabilities of each offspring.
    # The array returned is the probability of DD, AD, and AA offspring respectively.
    matingTable = {
        ("AA", "AA"): [0.0, 0.0, 1.0],
        ("AA", "AD"): [1/2.0 * P, 1/2.0 - 1/2.0 * P, 1/2.0],
        ("AA", "DD"): [P, 1- P, 0.0],
        ("AD", "AD"): [1/4.0 + 1/2.0 * P, 1/2.0 - 1/2.0 * P, 1/4.0],
        ("AD", "DD"): [1/2.0  + 1/2.0 * P, 1/2.0 - 1/2.0 * P, 0],
        ("DD", "DD"): [1.0, 0.0, 0.0]
    }

    # The table above only includes 6 of the 9 possible matings,
    # since we don't care about order.
    # That raises errors, for ones that are in different orders.
    # (E.g. (AD,DD) is in the dic, so (DD, AD) raises an error.)
    # Not sure how to cleanly deal with that, so we just catch the error.

    try:
        probs = matingTable[(A, B)]
    except KeyError:
        probs = matingTable[(B, A)]        
    offspring = np.random.choice(possibleOffspring, size=1, p = probs)[0]
    return offspring


# Takes in the array of genotypes, and the number of individuals in the population.

def runGeneration(G):
    N = G.numNodes
    totalFitness = findTotalFitness(G.genotypes)
    matingProbs = {}
    # Go through the upper-right triangle of the matrix:
    for i in range(N):
        for j in range(i+1, N):
            matingProb = fitness[G.genotypes[i]]*fitness[G.genotypes[j]]*G.weights[i][j]
            matingProbs[(i, j)] = matingProb

    # Now we have a dictionary that matches every pair with its probability of being a mate.
    # Not normalized probability though.

    # Choose a random pair:

    normalizedMatingProbs = np.array(matingProbs.values())/sum(matingProbs.values())
    index = np.random.choice(range(len(matingProbs.keys())), size=1, p = normalizedMatingProbs)[0]
    iRandMate, jRandMate = matingProbs.keys()[index]

    childGenotype = matingOutcome(G.genotypes[iRandMate], G.genotypes[jRandMate])

    # Find the neighbor to die:
    deathProbs = {}

    possibleForDeath = range(N)

    # Can't replace the parents.
    possibleForDeath.remove(iRandMate)
    possibleForDeath.remove(jRandMate)

    for k in possibleForDeath:
        deathProbs[k] = deathRates[G.genotypes[k]] * (G.weights[iRandMate][k] + G.weights[jRandMate][k])
    normalizedDeathProbs = np.array(deathProbs.values())/sum(deathProbs.values())
    toDie = np.random.choice(possibleForDeath, size=1, p = normalizedDeathProbs)[0]
    G.replaceNode(toDie, childGenotype)


    return

# Taking in a dict where the keys are diploid genotypes, e.g. "AD".
# We want to see how many many "D"s there are.
def getDriveAlleleFreq(G):
    freq = 0.0
    for geno in G.genotypeCounts:
        if geno == "AD":
            freq += 1 * G.genotypeCounts[geno]
        if geno == "DD":
            freq += 2 * G.genotypeCounts[geno]
    return freq/(2*G.numNodes)

def oneSimulation(G, indicesOfInitalDrive):
    # Start Drive:
    for i in indicesOfInitalDrive:
        G.replaceNode(i, "DD")
    DFreqs = []
    DFreq = -1.0
    numGens = 0
    while(DFreq != 0.0 and DFreq != 1.0):
        sys.stdout.flush()
        print "Current Gen: {}\r".format(numGens),
        numGens += 1
        runGeneration(G)
        DFreq = getDriveAlleleFreq(G)
        DFreqs.append(DFreq)
    Fixed = DFreq == 1.0
    return DFreqs, numGens, Fixed

def manySimulations(G, indicesOfInitalDrive, numSims):
    numFixed = 0.0
    arrayOfDFreqs = []
    for i in range(numSims):
        graph = copy.deepcopy(G)
        print "Sim # {}\r".format(i)
        DFreqs, numGens, Fixed = oneSimulation(graph, indicesOfInitalDrive)         
        numFixed += Fixed
        arrayOfDFreqs.append(DFreqs)
        sys.stdout.flush()
    fixationRate = numFixed/numSims
    return fixationRate

In [47]:
## Plotting

def createLatticeGenotypeTimestep(L):
    genotypeXs = {}
    genotypeYs = {}
    genotypeColorKey = {}
    genotypeColorsArrayForPlotting = {}
    genotypesIndex = 0
    for r in range(L.numRows):
        for c in range(L.numCols):
            currentGenotype = L.genotypes[genotypesIndex]
            # If we haven't seen this genotype yet, add entries in all the dictionaries.
            if currentGenotype not in genotypeXs:
                genotypeXs[currentGenotype] = []
                genotypeYs[currentGenotype] = []
                genotypeColorKey[currentGenotype] = 'rgb({}, {}, {})'.format(random.randint(0,255), random.randint(0,255), random.randint(0,255))
                genotypeColorsArrayForPlotting = []
                
            genotypeXs[currentGenotype].append(c)
            genotypeYs[currentGenotype].append(r)
            # Seems like plotly requires things to be the same length. So we create white circles for the other genotypes
            for g in genotypeColorsArrayForPlotting:
                if g == currentGenotype:
                    genotypeColorsArrayForPlotting[g].append(genotypeColorKey[g]) 
                else:
                    genotypeColorsArrayForPlotting[g].append('rgb(0, 0, 0)')    
            
            genotypesIndex += 1
    data = []
    for genotype in genotypeXs:
        n = len(genotypeXs[genotype])
        data.append({'x': genotypeXs[genotype], 'y': genotypeYs[genotype], 'name': genotype, 'mode': 'markers', 'marker':
            {'color': [genotypeColors[genotype]]*n,'size': [20]*n} })
    
    return data

In [50]:
L = Lattice(2,2)
timestep1 = createLatticeGenotypeTimestep(L)
L.replaceNode(1, "DD")
timestep2 = createLatticeGenotypeTimestep(L)
L.replaceNode(3, "DD")
timestep3 = createLatticeGenotypeTimestep(L)
timesteps = [timestep1, timestep2, timestep3]

print timestep1

[{'y': [0, 0, 1, 1], 'x': [0, 1, 0, 1], 'mode': 'markers', 'marker': {'color': ['rgb(198, 43, 171)', 'rgb(198, 43, 171)', 'rgb(198, 43, 171)', 'rgb(198, 43, 171)'], 'size': [20, 20, 20, 20]}}]


In [53]:
frames = []
for t in timesteps:
    frames.append({'data': t})
figure = {'data': [{'y': [0, 0, 1, 1], 'x': [0, 1, 0, 1], 'name': 'AA', 'marker': {'color': ['rgb(122, 189, 85)', 'rgb(122, 189, 85)', 'rgb(122, 189, 85)', 'rgb(122, 189, 85)'], 'size': [20, 20, 20, 20]}, 'mode': 'markers'}],
          'layout': {'xaxis': {'range': [0, 5], 'autorange': False},
                     'yaxis': {'range': [0, 5], 'autorange': False},
                     'title': 'Start Title',
                     'updatemenus': [{'type': 'buttons',
                                      'buttons': [{'label': 'Play',
                                                   'method': 'animate',
                                                   'args': [None]},
                                                  {
                                                    'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                                                    'transition': {'duration': 0}}],
                                                    'label': 'Pause',
                                                    'method': 'animate'
                                                }]}]
                    },
          'frames': frames}
   

print frames
figure['data'] = frames[0]
figure['frames'] = frames

# iplot(figure)

[{'data': [{'y': [0, 0, 1, 1], 'x': [0, 1, 0, 1], 'mode': 'markers', 'marker': {'color': ['rgb(198, 43, 171)', 'rgb(198, 43, 171)', 'rgb(198, 43, 171)', 'rgb(198, 43, 171)'], 'size': [20, 20, 20, 20]}}]}, {'data': [{'y': [0, 0, 1, 1], 'x': [0, 1, 0, 1], 'mode': 'markers', 'marker': {'color': ['rgb(45, 4, 32)', 'rgb(50, 203, 48)', 'rgb(45, 4, 32)', 'rgb(45, 4, 32)'], 'size': [20, 20, 20, 20]}}]}, {'data': [{'y': [0, 0, 1, 1], 'x': [0, 1, 0, 1], 'mode': 'markers', 'marker': {'color': ['rgb(97, 140, 133)', 'rgb(23, 96, 97)', 'rgb(97, 140, 133)', 'rgb(23, 96, 97)'], 'size': [20, 20, 20, 20]}}]}]


In [52]:
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML

init_notebook_mode(connected=True)

figure = {'data': [{'y': [0, 1, 1], 'x': [1, 0, 1], 'name': 'AA', 'marker': {'color': ['rgb(231, 47, 29)', 'rgb(231, 47, 29)', 'rgb(231, 47, 29)'], 'size': [20, 20, 20]}, 'mode': 'markers'}, {'y': [0], 'x': [0], 'name': 'DD', 'marker': {'color': ['rgb(18, 177, 47)'], 'size': [20]}, 'mode': 'markers'}],
          'layout': {'xaxis': {'range': [0, 5], 'autorange': False},
                     'yaxis': {'range': [0, 5], 'autorange': False},
                     'title': 'Start Title',
                     'updatemenus': [{'type': 'buttons',
                                      'buttons': [{'label': 'Play',
                                                   'method': 'animate',
                                                   'args': [None]},
                                                  {
                                                    'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                                                    'transition': {'duration': 0}}],
                                                    'label': 'Pause',
                                                    'method': 'animate'
                                                }]}]
                    },
          'frames': [{'data': [{'y': [0, 1, 1], 'x': [1, 0, 1], 'name': 'AA', 'marker': {'color': ['rgb(231, 47, 29)', 'rgb(231, 47, 29)', 'rgb(231, 47, 29)'], 'size': [20, 20, 20]}, 'mode': 'markers'}, {'y': [0], 'x': [0], 'name': 'DD', 'marker': {'color': ['rgb(18, 177, 47)'], 'size': [20]}, 'mode': 'markers'}]},
                     {'data': [{'y': [1, 1], 'x': [0, 1], 'name': 'AA', 'marker': {'color': ['rgb(149, 217, 103)', 'rgb(149, 217, 103)'], 'size': [20, 20]}, 'mode': 'markers'}, {'y': [0, 0], 'x': [0, 1], 'name': 'DD', 'marker': {'color': ['rgb(195, 227, 73)', 'rgb(195, 227, 73)'], 'size': [20, 20]}, 'mode': 'markers'}]},
                     {'data': [{'y': [1], 'x': [0], 'name': 'AA', 'marker': {'color': ['rgb(153, 35, 77)'], 'size': [20]}, 'mode': 'markers'}, {'y': [0, 0, 1], 'x': [0, 1, 1], 'name': 'DD', 'marker': {'color': ['rgb(11, 118, 241)', 'rgb(11, 118, 241)', 'rgb(11, 118, 241)'], 'size': [20, 20, 20]}, 'mode': 'markers'}]}]}
          
iplot(figure)

---